In [ ]:
!pip install plotly
!pip install tifffile
!pip install openpyxl
!pip install kaleido==0.2.1
!pip install imagecodecs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 67.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set
import sys
import tifffile
import plotly.figure_factory as ff
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
from google.colab import files

import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
COMPONENTES = ['FB', 'FF', 'NC', 'DC', 'LUMEN', 'MEDIA']

# Mapeamento das cores BGR para cada componente.
MAPA_CORES_BGR = {
    'FB': (0, 128, 0),        # Verde escuro
    'FF': (0, 255, 128),      # Verde-amarelado
    'NC': (0, 0, 255),        # Vermelho
    'DC': (255, 255, 255),    # Branco
    'LUMEN': (0, 0, 0),       # Preto
    'MEDIA': (128, 128, 128)  # Cinza
}

# Tolerância para ser 'FB'.
TOLERANCIA_COR = 30

def criar_mascaras_binarias(quadro_vh_bgr: np.ndarray) -> Dict[str, np.ndarray]:

    altura, largura = quadro_vh_bgr.shape[:2]

    mapa_de_mascaras = {comp: np.zeros((altura, largura), dtype=np.uint8)
                       for comp in COMPONENTES}

    distancias = np.zeros((altura, largura, len(COMPONENTES)), dtype=np.float32)

    quadro_float = quadro_vh_bgr.astype(np.float32)

    for i, (componente, cor_alvo_bgr) in enumerate(MAPA_CORES_BGR.items()):
        cor_alvo_float = np.array(cor_alvo_bgr, dtype=np.float32)

        diferenca = quadro_float - cor_alvo_float
        dist_quadrada = np.sum(diferenca ** 2, axis=2)
        distancias[:, :, i] = np.sqrt(dist_quadrada)

    # Encontra o componente mais próximo para cada pixel
    indices_componente_mais_proximo = np.argmin(distancias, axis=2)

    for i, componente in enumerate(COMPONENTES):
        mapa_de_mascaras[componente][indices_componente_mais_proximo == i] = 255

    return mapa_de_mascaras # Dicionário onde as chaves são os nomes dos componentes e os valores são as máscaras binárias (Altura x Largura)

In [ ]:
# LABELLING
# Implementa o algoritmo "Two-Pass" (Duas Passagens) com "Union-Find"
# para rotulagem de componentes conectados.

class UnionFind:
    """
    Estrutura de dados auxiliar para o algoritmo Two-Pass,
    usada para gerenciar equivalências entre rótulos temporários.
    """
    def __init__(self, n: int):
        self.pai = list(range(n))
        self.rank = [0] * n

    def find(self, i: int) -> int:
        """Encontra o rótulo raiz de 'i' com compressão de caminho."""
        if self.pai[i] == i:
            return i
        self.pai[i] = self.find(self.pai[i])
        return self.pai[i]

    def union(self, i: int, j: int):
        """Define que os rótulos 'i' e 'j' são equivalentes."""
        raiz_i = self.find(i)
        raiz_j = self.find(j)
        if raiz_i != raiz_j:
            if self.rank[raiz_i] < self.rank[raiz_j]:
                self.pai[raiz_i] = raiz_j
            elif self.rank[raiz_i] > self.rank[raiz_j]:
                self.pai[raiz_j] = raiz_i
            else:
                self.pai[raiz_j] = raiz_i
                self.rank[raiz_i] += 1

def rotular_componentes_conectados(mascara_binaria: np.ndarray, conectividade: int = 8
                                  ) -> Tuple[int, np.ndarray, np.ndarray, np.ndarray]:
    """
    Rotula componentes conectados em uma máscara binária usando o algoritmo Two-Pass.

    Args:
        mascara_binaria: Imagem preto e branco (0 ou 255)
        conectividade: 8 (inclui diagonais) ou 4 (apenas horizontal/vertical)

    Returns:
        - num_rotulos_final: O número total de ilhas encontradas.
        - mapa_de_rotulos: Imagem onde cada pixel tem o ID da sua ilha (0=fundo).
        - estatisticas: Array [N x 5] com [x, y, largura, altura, area] para cada rótulo.
        - centroides: Array [N x 2] com [cx, cy] para cada rótulo.
    """
    altura, largura = mascara_binaria.shape

    mapa_de_rotulos = np.zeros((altura, largura), dtype=np.int32)
    uf_equivalencias = UnionFind(altura * largura // 2)

    proximo_rotulo_disponivel = 1

    # Vizinhos a serem checados (apenas para trás na varredura)
    vizinhos_offsets = [
        (-1, -1), (-1,  0), (-1,  1), ( 0, -1)
    ]

    # Primeira Passagem: Atribuição de rótulos temporários e registro de equivalências
    for y in range(altura):
        for x in range(largura):
            if mascara_binaria[y, x] == 0:
                continue

            rotulos_vizinhos = set()
            for dy, dx in vizinhos_offsets:
                ny, nx = y + dy, x + dx
                if 0 <= ny < altura and 0 <= nx < largura and mapa_de_rotulos[ny, nx] > 0:
                    rotulos_vizinhos.add(mapa_de_rotulos[ny, nx])

            if not rotulos_vizinhos:
                mapa_de_rotulos[y, x] = proximo_rotulo_disponivel
                proximo_rotulo_disponivel += 1
            else:
                rotulo_minimo = min(rotulos_vizinhos)
                mapa_de_rotulos[y, x] = rotulo_minimo
                for r in rotulos_vizinhos:
                    if r != rotulo_minimo:
                        uf_equivalencias.union(rotulo_minimo, r)

    # Mapeamento de Rótulos: Resolve as equivalências temporárias para rótulos finais
    mapa_rotulo_final = {}
    num_rotulos_final = 1

    for r_temp in range(1, proximo_rotulo_disponivel):
        raiz = uf_equivalencias.find(r_temp)
        if raiz not in mapa_rotulo_final:
            mapa_rotulo_final[raiz] = num_rotulos_final
            num_rotulos_final += 1

    # Segunda Passagem: Aplica rótulos finais e coleta estatísticas
    areas = np.zeros(num_rotulos_final, dtype=np.int32)
    soma_x = np.zeros(num_rotulos_final, dtype=np.float64)
    soma_y = np.zeros(num_rotulos_final, dtype=np.float64)
    min_x = np.full(num_rotulos_final, largura, dtype=np.int32)
    max_x = np.full(num_rotulos_final, -1, dtype=np.int32)
    min_y = np.full(num_rotulos_final, altura, dtype=np.int32)
    max_y = np.full(num_rotulos_final, -1, dtype=np.int32)

    for y in range(altura):
        for x in range(largura):
            rotulo_temp = mapa_de_rotulos[y, x]
            if rotulo_temp > 0:
                raiz = uf_equivalencias.find(rotulo_temp)
                rotulo_final = mapa_rotulo_final[raiz]
                mapa_de_rotulos[y, x] = rotulo_final

                areas[rotulo_final] += 1
                soma_x[rotulo_final] += x
                soma_y[rotulo_final] += y
                min_x[rotulo_final] = min(min_x[rotulo_final], x)
                max_x[rotulo_final] = max(max_x[rotulo_final], x)
                min_y[rotulo_final] = min(min_y[rotulo_final], y)
                max_y[rotulo_final] = max(max_y[rotulo_final], y)

    estatisticas = np.zeros((num_rotulos_final, 5), dtype=np.int32)
    centroides = np.zeros((num_rotulos_final, 2), dtype=np.float64)

    for r in range(1, num_rotulos_final):
        if areas[r] > 0:
            estatisticas[r, 0] = min_x[r]
            estatisticas[r, 1] = min_y[r]
            estatisticas[r, 2] = (max_x[r] - min_x[r]) + 1
            estatisticas[r, 3] = (max_y[r] - min_y[r]) + 1
            estatisticas[r, 4] = areas[r]

            centroides[r, 0] = soma_x[r] / areas[r]
            centroides[r, 1] = soma_y[r] / areas[r]

    return num_rotulos_final, mapa_de_rotulos, estatisticas, centroides

def calcular_area_total(mascara_binaria: np.ndarray) -> int:
    """Calcula a área total (em pixels) de uma máscara binária."""
    return np.count_nonzero(mascara_binaria)

def filtrar_ilhas_por_tamanho(mascara_binaria: np.ndarray,
                             tamanho_minimo: int = 5) -> np.ndarray:
    """
    Remove ilhas menores que 'tamanho_minimo' pixels usando rotulagem manual.
    """
    num_rotulos, mapa_rotulos, estatisticas, _ = rotular_componentes_conectados(mascara_binaria)

    mascara_filtrada = np.zeros_like(mascara_binaria)

    for rotulo_id in range(1, num_rotulos):
        area = estatisticas[rotulo_id, 4]
        if area >= tamanho_minimo:
            mascara_filtrada[mapa_rotulos == rotulo_id] = 255

    return mascara_filtrada

def obter_mascaras_das_ilhas(mascara_binaria: np.ndarray,
                           tamanho_minimo: int = 0) -> Dict[int, np.ndarray]:
    """
    Retorna um dicionário onde cada item é a máscara binária de uma única ilha.
    """
    num_rotulos, mapa_rotulos, estatisticas, _ = rotular_componentes_conectados(mascara_binaria)

    mapa_mascaras_ilhas = {}

    for rotulo_id in range(1, num_rotulos):
        area = estatisticas[rotulo_id, 4]
        if area >= tamanho_minimo:
            mascara_ilha = np.zeros_like(mascara_binaria)
            mascara_ilha[mapa_rotulos == rotulo_id] = 255
            mapa_mascaras_ilhas[rotulo_id] = mascara_ilha

    return mapa_mascaras_ilhas

In [ ]:
# --- Análise e Medição---
# Contém a lógica para análises morfológicas, usando o bloco de labelling e OpenCV.

# Define o Elemento Estruturante (Kernel) para operações morfológicas.
ELEMENTO_ESTRUTURANTE_3x3 = np.ones((3, 3), dtype=np.uint8)

# --- Análise de Áreas Totais ---

def medir_areas_por_quadro(mapa_de_mascaras: Dict[str, np.ndarray]) -> Dict[str, int]:
    """
    Calcula a área total (em pixels) para cada componente em um quadro.
    """
    areas = {}
    for componente, mascara in mapa_de_mascaras.items():
        areas[componente] = calcular_area_total(mascara)
    return areas

# --- Análise de NC@DC ---

def calcular_area_nc_conectado_dc(mascara_nc: np.ndarray, mascara_dc: np.ndarray) -> int:
    """
    Calcula a área total das ilhas de NC que tocam em ilhas de DC.
    Usa dilatação para verificar a proximidade.
    """
    if not np.any(mascara_nc) or not np.any(mascara_dc):
        return 0

    num_rotulos, mapa_rotulos_nc, estatisticas_nc, _ = rotular_componentes_conectados(mascara_nc)

    rotulos_que_tocam_dc = set()

    for rotulo_id in range(1, num_rotulos):
        mascara_ilha_atual = (mapa_rotulos_nc == rotulo_id).astype(np.uint8) * 255
        # Aplica Dilatação para expandir a ilha
        ilha_dilatada = cv2.dilate(mascara_ilha_atual, ELEMENTO_ESTRUTURANTE_3x3, iterations=1)

        # Verifica intersecção com a máscara de DC
        if np.any(np.logical_and(ilha_dilatada, mascara_dc)):
            rotulos_que_tocam_dc.add(rotulo_id)

    area_total_nc_conectado_dc = 0
    for rotulo_id in rotulos_que_tocam_dc:
        area_original_da_ilha = estatisticas_nc[rotulo_id, 4]
        area_total_nc_conectado_dc += area_original_da_ilha

    return area_total_nc_conectado_dc

# --- Coleta de Intensidades Interiores ---

def coletar_intensidades_interiores(mascara_componente: np.ndarray,
                                    quadro_cinza: np.ndarray,
                                    tamanho_minimo_ilha: int = 5) -> np.ndarray:
    """
    Coleta os valores de intensidade do quadro em escala de cinza
    apenas dos pixels interiores de ilhas com tamanho mínimo.
    Usa erosão para encontrar os pixels interiores.
    """

    mascara_filtrada = filtrar_ilhas_por_tamanho(mascara_componente, tamanho_minimo=tamanho_minimo_ilha)
    if not np.any(mascara_filtrada):
        return np.array([], dtype=np.uint8)

    mapa_mascaras_ilhas = obter_mascaras_das_ilhas(mascara_filtrada, tamanho_minimo=tamanho_minimo_ilha)

    uniao_interiores = np.zeros_like(mascara_componente)

    for rotulo_id, mascara_ilha in mapa_mascaras_ilhas.items():
        # Aplica Erosão para remover a borda
        interior = cv2.erode(mascara_ilha, ELEMENTO_ESTRUTURANTE_3x3, iterations=1)
        uniao_interiores = np.bitwise_or(uniao_interiores, interior)

    intensidades = quadro_cinza[uniao_interiores > 0]

    return intensidades

def calcular_histograma_256(intensidades: np.ndarray) -> np.ndarray:
    """Calcula um histograma de 256 bins (0-255)."""
    if len(intensidades) == 0:
        return np.zeros(256, dtype=np.int64)
    hist, _ = np.histogram(intensidades, bins=256, range=(0, 256))
    return hist

# --- Funções Específicas para NC@DC ---

def criar_mascara_nc_conectado_dc(mascara_nc: np.ndarray, mascara_dc: np.ndarray) -> np.ndarray:
    """
    Cria uma máscara binária contendo apenas as ilhas de NC que tocam DC.
    Similar à função de cálculo de área, mas retorna a máscara.
    """
    if not np.any(mascara_nc) or not np.any(mascara_dc):
        return np.zeros_like(mascara_nc)

    num_rotulos, mapa_rotulos_nc, _, _ = rotular_componentes_conectados(mascara_nc)

    mascara_final_nc_conectado_dc = np.zeros_like(mascara_nc)

    for rotulo_id in range(1, num_rotulos):
        mascara_ilha_atual = (mapa_rotulos_nc == rotulo_id).astype(np.uint8) * 255
        ilha_dilatada = cv2.dilate(mascara_ilha_atual, ELEMENTO_ESTRUTURANTE_3x3, iterations=1)

        if np.any(np.logical_and(ilha_dilatada, mascara_dc)):
            mascara_final_nc_conectado_dc = np.bitwise_or(mascara_final_nc_conectado_dc, mascara_ilha_atual)

    return mascara_final_nc_conectado_dc

In [ ]:
# --- Classificador de Placas ---
# Implementa a classificação do tipo de placa baseada em critérios definidos.

def classificar_placa_quadro_unico(areas_componentes: Dict[str, float]) -> str:
    """
    Classifica o tipo de placa aterosclerótica para um único quadro.

    Args:
        areas_componentes: Dicionário com as áreas dos componentes.

    Returns:
        Uma string com o tipo da placa (ex: 'FibCa', 'VH-TCFA', 'PIT').
    """

    area_total_placa = (areas_componentes['fb_area'] + areas_componentes['ff_area'] +
                       areas_componentes['nc_area'] + areas_componentes['dc_area'])

    if area_total_placa == 0:
        return 'SEM_PLACA'

    perc_nc = (areas_componentes['nc_area'] / area_total_placa) * 100
    perc_fb = (areas_componentes['fb_area'] / area_total_placa) * 100
    perc_dc = (areas_componentes['dc_area'] / area_total_placa) * 100
    perc_ff = (areas_componentes['ff_area'] / area_total_placa) * 100

    # Regras de classificação baseadas em Maehara A, et al.
    if perc_nc >= 10:
        taxa_nc_confluente = 0
        if areas_componentes['nc_area'] > 0:
            # Proxy para NC confluente/exposto
            taxa_nc_confluente = areas_componentes['nc@dc_area'] / areas_componentes['nc_area']

        if taxa_nc_confluente > 0.1:
            return 'VH-TCFA'
        else:
            return 'ThCFA'

    if perc_ff > 30 and perc_nc < 10 and perc_dc < 10:
        return 'PIT'

    if perc_dc > 10 and perc_nc < 10:
        return 'FibCa'

    if perc_nc < 10 and perc_dc < 10 and perc_ff < 15:
        if perc_fb > perc_ff:
            return 'Fibrotic'

    if perc_fb > 50:
        return 'Fibrotic'

    return 'PIT'


def classificar_placas_todos_quadros(lista_medidas_quadros: List[Dict]) -> List[str]:
    """Aplica o classificador em todos os quadros."""
    lista_classificacoes = []
    for medidas in lista_medidas_quadros:
        areas_para_classificar = {
            'fb_area': medidas['FB'],
            'ff_area': medidas['FF'],
            'nc_area': medidas['NC'],
            'dc_area': medidas['DC'],
            'nc@dc_area': medidas['NC_@_DC']
        }
        tipo_placa = classificar_placa_quadro_unico(areas_para_classificar)
        lista_classificacoes.append(tipo_placa)
    return lista_classificacoes


def imprimir_relatorio_classificacao(lista_classificacoes: List[str]):
    """Imprime um resumo da contagem de cada tipo de placa encontrada."""
    print("\n--- Relatório de Classificação de Placas ---")
    tipos_unicos, contagens = np.unique(lista_classificacoes, return_counts=True)
    resumo_contagem = dict(zip(tipos_unicos, contagens))
    total_quadros = len(lista_classificacoes)

    print(f"Total de quadros analisados: {total_quadros}")
    print("Distribuição de tipos de placa:")

    tipos_ordenados = sorted(resumo_contagem.items(), key=lambda item: item[1], reverse=True)

    for tipo_placa, contagem in tipos_ordenados:
        porcentagem = (contagem / total_quadros) * 100
        print(f"  {tipo_placa:10s}: {contagem:4d} quadros ({porcentagem:5.1f}%)")
    print("-" * 50)

In [ ]:
# Contém funções para ler e escrever arquivos TIFF e Excel.

# --- Leitura ---

def carregar_stack_tiff(caminho_arquivo: str) -> List[np.ndarray]:
    """
    Carrega um arquivo TIFF multi-página e retorna uma lista de frames.
    Tenta usar cv2.imreadmulti e tifffile como fallback.
    """
    caminho = Path(caminho_arquivo)
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")

    sucesso, lista_quadros = cv2.imreadmulti(str(caminho), flags=cv2.IMREAD_UNCHANGED)

    if not sucesso or not lista_quadros:
        print(f"Aviso: cv2.imreadmulti falhou. Tentando com tifffile...")
        try:
            array_tiff = tifffile.imread(str(caminho))
            if array_tiff.ndim == 3:
                lista_quadros = list(array_tiff)
            elif array_tiff.ndim == 2:
                lista_quadros = [array_tiff]
            else:
                 raise ValueError("Formato TIFF inesperado.")
        except Exception as e:
            raise ValueError(f"Erro ao ler o arquivo TIFF com cv2 e tifffile: {caminho}. Erro: {e}")

    if not lista_quadros:
        raise ValueError(f"Arquivo TIFF está vazio ou é inválido: {caminho}")

    return lista_quadros


def carregar_stacks_vh_gs(caminho_vh: str, caminho_gs: str
                         ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Carrega os arquivos VH e GS, valida a compatibilidade e garante
    os formatos de cor corretos (VH=BGR, GS=GRAY).
    """
    print("Carregando stack VH-IVUS...")
    quadros_vh_raw = carregar_stack_tiff(caminho_vh)

    print("Carregando stack GS-IVUS...")
    quadros_gs_raw = carregar_stack_tiff(caminho_gs)

    if len(quadros_vh_raw) != len(quadros_gs_raw):
        raise ValueError(
            f"Número de quadros não coincide: "
            f"VH={len(quadros_vh_raw)}, GS={len(quadros_gs_raw)}"
        )

    shape_vh = quadros_vh_raw[0].shape[:2]
    shape_gs = quadros_gs_raw[0].shape[:2]
    if shape_vh != shape_gs:
        raise ValueError(
            f"Dimensões dos quadros não coincidem: "
            f"VH={shape_vh}, GS={shape_gs}"
        )

    quadros_vh_bgr = []
    for quadro in quadros_vh_raw:
        if quadro.ndim == 2:
            quadros_vh_bgr.append(cv2.cvtColor(quadro, cv2.COLOR_GRAY2BGR))
        elif quadro.ndim == 3 and quadro.shape[2] == 4:
            quadros_vh_bgr.append(cv2.cvtColor(quadro, cv2.COLOR_RGBA2BGR))
        elif quadro.ndim == 3 and quadro.shape[2] == 3:
            quadros_vh_bgr.append(quadro)
        else:
            raise ValueError(f"Formato de quadro VH inesperado: {quadro.shape}")

    quadros_gs_cinza = []
    for quadro in quadros_gs_raw:
        if quadro.ndim == 3:
            quadros_gs_cinza.append(cv2.cvtColor(quadro, cv2.COLOR_BGR2GRAY))
        elif quadro.ndim == 2:
            quadros_gs_cinza.append(quadro)
        else:
            raise ValueError(f"Formato de quadro GS inesperado: {quadro.shape}")

    print(f"Validação concluída: {len(quadros_vh_bgr)} quadros (dimensões {shape_vh}) carregados.")
    return quadros_vh_bgr, quadros_gs_cinza

# --- Escrita ---

def salvar_stack_tiff(lista_quadros: List[np.ndarray], caminho_saida: Path):
    """Salva uma lista de quadros como um único arquivo TIFF multi-página."""
    stack = np.array(lista_quadros, dtype=np.uint8)
    tifffile.imwrite(str(caminho_saida), stack, compression='lzw')

def salvar_graficos_histogramas(mapa_histogramas: Dict[str, np.ndarray], diretorio_saida: Path):
    """
    Cria e salva os gráficos de histograma para cada componente.
    """
    print("\n--- Gerando gráficos de histogramas ---")
    mapa_cores = {
        'FB': 'green', 'FF': 'yellowgreen', 'NC': 'red', 'DC': 'lightgray',
        'LUMEN': 'black', 'MEDIA': 'gray', 'NC@DC': 'darkred'
    }

    for componente, hist in mapa_histogramas.items():
        plt.figure(figsize=(10, 6))
        bins = np.arange(256)
        plt.bar(bins, hist, width=1.0, color=mapa_cores.get(componente, 'blue'),
                alpha=0.7, edgecolor='none')

        plt.title(f'Histograma - Componente {componente}', fontsize=14)
        plt.xlabel('Intensidade (0-255)', fontsize=12)
        plt.ylabel('Frequência (Contagem de Pixels)', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.xlim(-0.5, 255.5)

        caminho_arquivo = diretorio_saida / f"histograma_{componente.lower()}.png"
        plt.savefig(str(caminho_arquivo), dpi=150, bbox_inches='tight')
        plt.close()
        print(f"  Salvo: {caminho_arquivo.name}")

def salvar_grafico_dendrograma(mapa_histogramas: Dict[str, np.ndarray], diretorio_saida: Path):
    """
    Cria e salva o gráfico de dendrograma para mostrar a similaridade entre histogramas.
    Normaliza histogramas, calcula distâncias e realiza agrupamento hierárquico.
    """
    print("\n--- Gerando dendrograma ---")

    lista_nomes_componentes = []
    lista_features_histogramas = []

    for componente, hist in mapa_histogramas.items():
        lista_nomes_componentes.append(componente)
        total = np.sum(hist)
        if total == 0:
            lista_features_histogramas.append(np.zeros(256, dtype=np.float64))
        else:
            lista_features_histogramas.append(hist.astype(np.float64) / total)

    matriz_features = np.array(lista_features_histogramas)

    metrica_distancia = 'euclidean'
    metodo_agrupamento = 'ward'

    fig = ff.create_dendrogram(
        matriz_features,
        labels=lista_nomes_componentes,
        linkagefun=lambda x: linkage(x, method=metodo_agrupamento, metric=metrica_distancia)
    )

    fig.update_layout(
        title='Dendrograma de Similaridade entre Componentes',
        xaxis_title='Componente',
        yaxis_title=f'Distância ({metrica_distancia})',
        width=1000, height=600
    )

    caminho_arquivo = diretorio_saida / "dendrograma.png"
    fig.write_image(str(caminho_arquivo))
    print(f"  Salvo: {caminho_arquivo.name}")

    fig.show()

def salvar_planilha_excel(df_medidas: pd.DataFrame,
                          df_histogramas: pd.DataFrame,
                          caminho_saida: Path):
    """
    Salva DataFrames de medidas e histogramas em um arquivo Excel com abas separadas.
    """
    print("\n--- Salvando planilha Excel ---")
    with pd.ExcelWriter(str(caminho_saida), engine='openpyxl') as writer:
        df_medidas.to_excel(writer, sheet_name='Medidas_Quadro_a_Quadro', index=False)
        df_histogramas.to_excel(writer, sheet_name='Histogramas_Dados', index=False)
    print(f"  Salvo: {caminho_saida.name}")


In [ ]:
# --- Funções Principais que serão chamadas pela main

def carregar_dados(caminho_vh: str, caminho_gs: str
                           ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Carrega os dados brutos (VH e GS) dos arquivos TIFF.
    """
    print("\n--- INICIANDO ETAPA 1: Carregar Dados ---")
    quadros_vh_bgr, quadros_gs_cinza = carregar_stacks_vh_gs(caminho_vh, caminho_gs)
    return quadros_vh_bgr, quadros_gs_cinza


def criar_mascaras(quadros_vh_bgr: List[np.ndarray]
                           ) -> List[Dict[str, np.ndarray]]:
    """
    Processa os quadros VH coloridos e os segmenta em máscaras binárias.
    """
    print("\n--- INICIANDO ETAPA 2: Criar Máscaras Binárias ---")
    lista_mascaras_todos_quadros = []
    total_quadros = len(quadros_vh_bgr)
    for i, quadro in enumerate(quadros_vh_bgr):
        if (i + 1) % 20 == 0 or i == 0:
            print(f"  Processando máscara para o quadro {i+1}/{total_quadros}...")
        mapa_de_mascaras_quadro = criar_mascaras_binarias(quadro)
        lista_mascaras_todos_quadros.append(mapa_de_mascaras_quadro)
    print(f"Máscaras criadas para {len(lista_mascaras_todos_quadros)} quadros.")
    return lista_mascaras_todos_quadros


def calcular_medidas(lista_mascaras_todos_quadros: List[Dict[str, np.ndarray]]
                             ) -> pd.DataFrame:
    """
    Calcula as áreas, NC@DC e classifica as placas para cada quadro.
    """
    print("\n--- INICIANDO ETAPA 3: Calcular Medidas ---")
    lista_resultados_medidas = []
    total_quadros = len(lista_mascaras_todos_quadros)
    for i, mapa_mascaras in enumerate(lista_mascaras_todos_quadros):
        if (i + 1) % 20 == 0 or i == 0:
            print(f"  Calculando medidas para o quadro {i+1}/{total_quadros}...")

        medidas_areas = medir_areas_por_quadro(mapa_mascaras)
        area_nc_dc = calcular_area_nc_conectado_dc(mapa_mascaras['NC'], mapa_mascaras['DC'])

        resultado_quadro = {
            'Quadro': i,
            'LUMEN': medidas_areas['LUMEN'],
            'MEDIA': medidas_areas['MEDIA'],
            'FB': medidas_areas['FB'],
            'FF': medidas_areas['FF'],
            'NC': medidas_areas['NC'],
            'DC': medidas_areas['DC'],
            'NC_AT_DC': area_nc_dc
        }
        lista_resultados_medidas.append(resultado_quadro)

    print("  Classificando tipos de placa...")
    lista_classificacoes = classificar_placas_todos_quadros(lista_resultados_medidas)
    imprimir_relatorio_classificacao(lista_classificacoes)

    df_medidas = pd.DataFrame(lista_resultados_medidas)
    df_medidas['Tipo_Placa'] = lista_classificacoes

    colunas_ordenadas = ['Quadro', 'Tipo_Placa', 'LUMEN', 'MEDIA',
                         'FB', 'FF', 'NC', 'DC', 'NC_AT_DC']
    df_medidas = df_medidas[colunas_ordenadas]

    return df_medidas


def calcular_histogramas(lista_mascaras_todos_quadros: List[Dict[str, np.ndarray]],
                                 quadros_gs_cinza: List[np.ndarray]
                                 ) -> Tuple[Dict[str, np.ndarray], pd.DataFrame]:
    """
    Calcula os histogramas de intensidade para os componentes em todos os quadros.
    """
    print("\n--- INICIANDO ETAPA 4: Calcular Histogramas ---")

    componentes_para_hist = ['FB', 'FF', 'NC', 'DC', 'LUMEN', 'MEDIA']
    mapa_histogramas_finais = {}
    acumulador_intensidades = {comp: [] for comp in componentes_para_hist + ['NC@DC']}
    total_quadros = len(lista_mascaras_todos_quadros)

    for i in range(total_quadros):
        if (i + 1) % 20 == 0 or i == 0:
            print(f"  Coletando intensidades do quadro {i+1}/{total_quadros}...")

        mapa_mascaras = lista_mascaras_todos_quadros[i]
        quadro_cinza = quadros_gs_cinza[i]

        for comp in componentes_para_hist:
            intensidades = coletar_intensidades_interiores(mapa_mascaras[comp], quadro_cinza, tamanho_minimo_ilha=5)
            if len(intensidades) > 0:
                acumulador_intensidades[comp].append(intensidades)

        mascara_nc_dc = criar_mascara_nc_conectado_dc(mapa_mascaras['NC'], mapa_mascaras['DC'])
        intensidades_nc_dc = coletar_intensidades_interiores(mascara_nc_dc, quadro_cinza, tamanho_minimo_ilha=5)
        if len(intensidades_nc_dc) > 0:
            acumulador_intensidades['NC@DC'].append(intensidades_nc_dc)

    print("  Calculando histogramas finais...")
    lista_para_df_hist = []

    for componente in acumulador_intensidades.keys():
        if not acumulador_intensidades[componente]:
            intensidades_finais = np.array([], dtype=np.uint8)
        else:
            intensidades_finais = np.concatenate(acumulador_intensidades[componente])

        hist_final = calcular_histograma_256(intensidades_finais)
        mapa_histogramas_finais[componente] = hist_final

        dados_linha = {'componente': componente, 'total_pixels_contados': np.sum(hist_final)}
        for bin_idx in range(256):
            dados_linha[f'bin_{bin_idx}'] = hist_final[bin_idx]
        lista_para_df_hist.append(dados_linha)

        print(f"    - {componente}: {np.sum(hist_final)} pixels interiores contados.")

    df_histogramas = pd.DataFrame(lista_para_df_hist)
    return mapa_histogramas_finais, df_histogramas


def salvar_resultados(diretorio_saida_base: Path,
                              df_medidas: pd.DataFrame,
                              mapa_histogramas: Dict[str, np.ndarray],
                              df_histogramas: pd.DataFrame,
                              lista_mascaras_todos_quadros: List[Dict[str, np.ndarray]]):
    """
    Salva todos os arquivos de saída (tabelas, gráficos, máscaras TIFF).
    """
    print("\n--- INICIANDO ETAPA 5: Salvar Resultados ---")

    dir_tabelas = diretorio_saida_base / "tabelas"
    dir_graficos = diretorio_saida_base / "graficos"
    dir_mascaras = diretorio_saida_base / "mascaras"

    dir_tabelas.mkdir(parents=True, exist_ok=True)
    dir_graficos.mkdir(parents=True, exist_ok=True)
    dir_mascaras.mkdir(parents=True, exist_ok=True)

    salvar_graficos_histogramas(mapa_histogramas, dir_graficos)
    salvar_grafico_dendrograma(mapa_histogramas, dir_graficos)

    print("\n--- Salvando máscaras TIFF ---")
    for componente in COMPONENTES:
        lista_quadros_componente = [
            mapa_mascaras_quadro[componente]
            for mapa_mascaras_quadro in lista_mascaras_todos_quadros
        ]
        caminho_saida_tiff = dir_mascaras / f"mascara_{componente.lower()}.tif"
        salvar_stack_tiff(lista_quadros_componente, caminho_saida_tiff)
        print(f"  Salvo: {caminho_saida_tiff.name}")

    caminho_planilha = dir_tabelas / "planilha_resultados_finais.xlsx"
    salvar_planilha_excel(df_medidas, df_histogramas, caminho_planilha)

    print("\n--- Resultados salvos ---")
    print(f"Tabelas: {dir_tabelas}")
    print(f"Graficos: {dir_graficos}")
    print(f"Máscaras: {dir_mascaras}")

In [ ]:
# --- Main ---

def main():
    """Função principal que executa todo o pipeline."""

    # 1. Define o caminho dos arquivos de entrada (VH e GS)
    CAMINHO_ARQUIVO_VH = "grupo5_VH.tif"
    CAMINHO_ARQUIVO_GS = "grupo5_GS.tif"

    # 2. Defina o nome do diretório de saída
    NOME_DIRETORIO_SAIDA = "results"

    diretorio_saida = Path(NOME_DIRETORIO_SAIDA)

    # 3. Validação de Arquivos
    if not Path(CAMINHO_ARQUIVO_VH).exists():
        print(f"❌ ERRO: Arquivo não encontrado: {CAMINHO_ARQUIVO_VH}")
        return
    if not Path(CAMINHO_ARQUIVO_GS).exists():
        print(f"❌ ERRO: Arquivo não encontrado: {CAMINHO_ARQUIVO_GS}")
        return

    print("Arquivos de entrada encontrados. Iniciando...")
    print("="*60)

    try:
        # ETAPA 1: Carregar Dados
        quadros_vh_bgr, quadros_gs_cinza = carregar_dados(
            CAMINHO_ARQUIVO_VH, CAMINHO_ARQUIVO_GS
        )

        # ETAPA 2: Criar Máscaras Binárias
        # Para cada pixel da imagem colorida, ela calcula a 'distância' da cor desse pixel para as 6 cores-alvo .
        # Ela então classifica o pixel com base na cor mais próxima.
        # O resultado é um conjunto de 6 máscaras binárias para cada. "

        lista_mascaras_todos_quadros = criar_mascaras(quadros_vh_bgr)

        # ETAPA 3: labelling
        df_medidas_finais = calcular_medidas(lista_mascaras_todos_quadros)

        # ETAPA 4: Calcular Histogramas
        mapa_histogramas_finais, df_histogramas_finais = calcular_histogramas(
            lista_mascaras_todos_quadros, quadros_gs_cinza
        )


        salvar_resultados(
            diretorio_saida,
            df_medidas_finais,
            mapa_histogramas_finais,
            df_histogramas_finais,
            lista_mascaras_todos_quadros
        )

        print("\n" + "="*60)
        print("✅ PIPELINE CONCLUÍDO COM SUCESSO!")
        print(f"Todos os resultados foram salvos no diretório: '{NOME_DIRETORIO_SAIDA}'")
        print("="*60)

    except Exception as e:
        print("\n" + "!"*60)
        print(f"❌ ERRO INESPERADO: O pipeline falhou.")
        print(f"Erro: {e}")
        import traceback
        traceback.print_exc()
        print("!"*60)

main()

Arquivos de entrada encontrados. Iniciando...

--- INICIANDO ETAPA 1: Carregar Dados ---
Carregando stack VH-IVUS...
Carregando stack GS-IVUS...
Validação concluída: 26 quadros (dimensões (400, 400)) carregados.

--- INICIANDO ETAPA 2: Criar Máscaras Binárias ---
  Processando máscara para o quadro 1/26...
  Processando máscara para o quadro 20/26...
Máscaras criadas para 26 quadros.

--- INICIANDO ETAPA 3: Calcular Medidas ---
  Calculando medidas para o quadro 1/26...
  Calculando medidas para o quadro 20/26...
  Classificando tipos de placa...

--- Relatório de Classificação de Placas ---
Total de quadros analisados: 26
Distribuição de tipos de placa:
  VH-TCFA   :   26 quadros (100.0%)
--------------------------------------------------

--- INICIANDO ETAPA 4: Calcular Histogramas ---
  Coletando intensidades do quadro 1/26...
  Coletando intensidades do quadro 20/26...
  Calculando histogramas finais...
    - FB: 43984 pixels interiores contados.
    - FF: 708 pixels interiores con


--- Salvando máscaras TIFF ---
  Salvo: mascara_fb.tif
  Salvo: mascara_ff.tif
  Salvo: mascara_nc.tif
  Salvo: mascara_dc.tif
  Salvo: mascara_lumen.tif
  Salvo: mascara_media.tif

--- Salvando planilha Excel ---
  Salvo: planilha_resultados_finais.xlsx

--- Resultados salvos ---
Tabelas: results/tabelas
Graficos: results/graficos
Máscaras: results/mascaras

✅ PIPELINE CONCLUÍDO COM SUCESSO!
Todos os resultados foram salvos no diretório: 'results'
